In [2]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.5f}'.format

In [3]:
POS_CASH_df = pd.read_csv('POS_CASH_balance.csv')

In [4]:
POS_CASH_df.info(verbose=True, null_counts=True)

C:\Users\TaeSoo\AppData\Local\Temp/ipykernel_25948/668184719.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  POS_CASH_df.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Non-Null Count     Dtype  
---  ------                 --------------     -----  
 0   SK_ID_PREV             10001358 non-null  int64  
 1   SK_ID_CURR             10001358 non-null  int64  
 2   MONTHS_BALANCE         10001358 non-null  int64  
 3   CNT_INSTALMENT         9975287 non-null   float64
 4   CNT_INSTALMENT_FUTURE  9975271 non-null   float64
 5   NAME_CONTRACT_STATUS   10001358 non-null  object 
 6   SK_DPD                 10001358 non-null  int64  
 7   SK_DPD_DEF             10001358 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


In [5]:
POS_CASH_df.describe(include='all')

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
count,10001358.00000,10001358.00000,10001358.00000,9975287.00000,9975271.00000,10001358,10001358.00000,10001358.00000
unique,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,9151119,NaN,NaN
mean,1903216.59896,278403.86331,-35.01259,17.08965,10.48384,NaN,11.60693,0.65447
std,535846.53072,102763.74509,26.06657,11.99506,11.10906,NaN,132.71404,32.76249
min,1000001.00000,100001.00000,-96.00000,1.00000,0.00000,NaN,0.00000,0.00000
25%,1434405.00000,189550.00000,-54.00000,10.00000,3.00000,NaN,0.00000,0.00000
50%,1896565.00000,278654.00000,-28.00000,12.00000,7.00000,NaN,0.00000,0.00000
75%,2368963.00000,367429.00000,-13.00000,24.00000,14.00000,NaN,0.00000,0.00000


In [6]:
def object_pre (df) :
    
    pk_id = sorted(list(df['SK_ID_PREV'].unique()))
    first_id = []
    mode_id = []
    object_col_first = []
    object_col_mode = []

    for col in df:
        if df[col].dtype == 'object':
            first_id.append(col+'_FIRST')
            mode_id.append(col+'_MODE')
            print(col)
            object_col_first.append(list(df.groupby('SK_ID_PREV')[col].first()))
            object_col_mode.append(list(df.groupby('SK_ID_PREV')[col].apply(lambda x : x.mode()[0])))

    object_col_first_df = pd.DataFrame(object_col_first).transpose()
    object_col_mode_df = pd.DataFrame(object_col_mode).transpose()   

    object_col_first_df.columns = [first_id]
    object_col_mode_df.columns = [mode_id]
    return pk_id, object_col_first_df, object_col_mode_df

In [7]:
pk_id, object_col_first, object_col_mode = object_pre(POS_CASH_df)

NAME_CONTRACT_STATUS


In [13]:
merge_obj_df = pd.concat([object_col_first, object_col_mode], axis = 1)
merge_obj_df.index = pk_id
merge_obj_df = merge_obj_df.reset_index()
merge_obj_df = merge_obj_df.rename(columns={'index':'SK_ID_PREV'})
merge_obj_df.columns = ['SK_ID_PREV', 'NAME_CONTRACT_STATUS_FIRST', 'NAME_CONTRACT_STATUS_MODE']
merge_obj_df

,SK_ID_PREV,NAME_CONTRACT_STATUS_FIRST,NAME_CONTRACT_STATUS_MODE
0,1000001,Completed,Active
1,1000002,Active,Active
2,1000003,Active,Active
3,1000004,Active,Active
4,1000005,Active,Active
...,...,...,...
936320,2843494,Completed,Active
936321,2843495,Completed,Active
936322,2843497,Active,Active
936323,2843498,Active,Active


In [14]:
level_0 = ['MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF']
level_1 = ['_MEAN', '_MEDIAN', '_STD', '_FIRST']

col_names = []
for name1 in level_0 :
    for name2 in level_1 :
        col_names.append(name1+name2)

In [15]:
merge_real_num_df = pd.DataFrame(POS_CASH_df.groupby('SK_ID_PREV').agg(['mean','median','std','first']))
merge_real_num_df.reset_index(drop = True, inplace = True)
merge_real_num_df.columns = merge_real_num_df.columns.droplevel(0)
merge_real_num_df = merge_real_num_df.iloc[:,4:]

merge_real_num_df.columns = col_names
merge_real_num_df

,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MEDIAN,MONTHS_BALANCE_STD,MONTHS_BALANCE_FIRST,CNT_INSTALMENT_MEAN,CNT_INSTALMENT_MEDIAN,CNT_INSTALMENT_STD,CNT_INSTALMENT_FIRST,CNT_INSTALMENT_FUTURE_MEAN,CNT_INSTALMENT_FUTURE_MEDIAN,CNT_INSTALMENT_FUTURE_STD,CNT_INSTALMENT_FUTURE_FIRST,SK_DPD_MEAN,SK_DPD_MEDIAN,SK_DPD_STD,SK_DPD_FIRST,SK_DPD_DEF_MEAN,SK_DPD_DEF_MEDIAN,SK_DPD_DEF_STD,SK_DPD_DEF_FIRST
0,-9.00000,-9.00000,1.00000,-8,8.66667,12.00000,5.77350,2.00000,7.66667,11.00000,6.65833,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
1,-52.00000,-52.00000,1.58114,-52,5.20000,6.00000,1.09545,6.00000,2.00000,2.00000,1.58114,2.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
2,-2.50000,-2.50000,1.29099,-3,12.00000,12.00000,0.00000,12.00000,10.50000,10.50000,1.29099,11.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
3,-25.50000,-25.50000,2.44949,-26,9.62500,10.00000,1.06066,10.00000,6.12500,6.50000,3.18198,7.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
4,-51.00000,-51.00000,3.31662,-49,10.00000,10.00000,0.00000,10.00000,5.00000,5.00000,3.31662,3.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936320,-25.00000,-25.00000,1.00000,-24,32.66667,48.00000,26.55811,2.00000,31.66667,47.00000,27.42870,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
936321,-12.50000,-12.50000,2.44949,-9,53.37500,60.00000,18.73833,7.00000,49.87500,56.50000,20.25154,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
936322,-11.00000,-11.00000,6.20484,-15,24.00000,24.00000,0.00000,24.00000,14.00000,14.00000,6.20484,18.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
936323,-45.00000,-45.00000,2.16025,-47,27.42857,36.00000,14.63850,36.00000,24.28571,33.00000,16.64046,35.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0


In [16]:
total_POS_CASH_df = pd.concat([merge_obj_df, merge_real_num_df], axis = 1)
total_POS_CASH_df

,SK_ID_PREV,NAME_CONTRACT_STATUS_FIRST,NAME_CONTRACT_STATUS_MODE,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MEDIAN,MONTHS_BALANCE_STD,MONTHS_BALANCE_FIRST,CNT_INSTALMENT_MEAN,CNT_INSTALMENT_MEDIAN,CNT_INSTALMENT_STD,...,CNT_INSTALMENT_FUTURE_STD,CNT_INSTALMENT_FUTURE_FIRST,SK_DPD_MEAN,SK_DPD_MEDIAN,SK_DPD_STD,SK_DPD_FIRST,SK_DPD_DEF_MEAN,SK_DPD_DEF_MEDIAN,SK_DPD_DEF_STD,SK_DPD_DEF_FIRST
0,1000001,Completed,Active,-9.00000,-9.00000,1.00000,-8,8.66667,12.00000,5.77350,...,6.65833,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
1,1000002,Active,Active,-52.00000,-52.00000,1.58114,-52,5.20000,6.00000,1.09545,...,1.58114,2.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
2,1000003,Active,Active,-2.50000,-2.50000,1.29099,-3,12.00000,12.00000,0.00000,...,1.29099,11.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
3,1000004,Active,Active,-25.50000,-25.50000,2.44949,-26,9.62500,10.00000,1.06066,...,3.18198,7.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
4,1000005,Active,Active,-51.00000,-51.00000,3.31662,-49,10.00000,10.00000,0.00000,...,3.31662,3.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936320,2843494,Completed,Active,-25.00000,-25.00000,1.00000,-24,32.66667,48.00000,26.55811,...,27.42870,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
936321,2843495,Completed,Active,-12.50000,-12.50000,2.44949,-9,53.37500,60.00000,18.73833,...,20.25154,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
936322,2843497,Active,Active,-11.00000,-11.00000,6.20484,-15,24.00000,24.00000,0.00000,...,6.20484,18.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0
936323,2843498,Active,Active,-45.00000,-45.00000,2.16025,-47,27.42857,36.00000,14.63850,...,16.64046,35.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0


In [17]:
total_POS_CASH_df.to_csv('POS_CASH_merge.csv')